In [2]:
from tvtk.api import tvtk, write_data
import pyvista as pv 
import os
import numpy as np

def pv_plot(B, vtk_path='./evaluation.vtk', points=((7, 64, 8), (7, 64, 8)), title='LL', overwrite=False):

    if not os.path.exists(vtk_path) or overwrite:
        dim = B.shape[:-1]
        pts = np.stack(np.mgrid[0:dim[0], 0:dim[1], 0:dim[2]], -1).astype(np.float32)
        pts = pts.transpose(2, 1, 0, 3)
        pts = pts.reshape((-1, 3))
        vectors = B.transpose(2, 1, 0, 3)
        vectors = vectors.reshape((-1, 3))
        sg = tvtk.StructuredGrid(dimensions=dim, points=pts)
        sg.point_data.vectors = vectors
        sg.point_data.vectors.name = 'B'
        write_data(sg, str(vtk_path))

    mesh = pv.read(vtk_path)
    xindmax, yindmax, zindmax = mesh.dimensions
    xcenter, ycenter, zcenter = mesh.center

    p = pv.Plotter()
    p.add_mesh(mesh.outline())

    sargs_B = dict(
        title='Bz [G]',
        title_font_size=15,
        height=0.25,
        width=0.05,
        vertical=True,
        position_x = 0.05,
        position_y = 0.05,
    )
    dargs_B = dict(
        scalars='B', 
        component=2, 
        clim=(-150, 150), 
        scalar_bar_args=sargs_B, 
        show_scalar_bar=False, 
        lighting=False
    )
    p.add_mesh(mesh.extract_subset((0, xindmax, 0, yindmax, 0, 0)), 
            cmap='gray', **dargs_B)

    def draw_streamlines(pts):
        stream, src = mesh.streamlines(
            return_source=True,
            start_position = pts,
            integration_direction='both',
            max_time=1000,
        )
        # print(pts)
        key = pts[0]*pts[1] + (pts[0]//pts[1]) + (pts[0] - pts[1])
        # print(key)
        np.random.seed(key)
        colors = np.random.rand(3)
        # if pts[0] == 16 and pts[1] == 48:
        #     colors = 'white'
        # print(colors)
        p.add_mesh(stream.tube(radius=0.2), lighting=False, color=colors)
        p.add_mesh(src, point_size=7, color=colors)

    xrange = points[0]
    yrange = points[1]
    for i in np.arange(*xrange):
        for j in np.arange(*yrange):
            try: 
                draw_streamlines((i, j, 0))
            except:
                print(i, j)

    p.camera_position = 'xy'
    p.show_bounds()
    # p.add_title(title)

    return p

In [3]:
import numpy as np 
import pandas as pd

from pinf.metric import vector_norm, curl, divergence

def metrics(B, b, B_potential, b_potential):
    """
    B is the numerical solution
    b is the reference magnetic field
    """

    c_vec = np.sum((B * b).sum(-1)) / np.sqrt((B ** 2).sum(-1).sum() * (b ** 2).sum(-1).sum())

    M = np.prod(B.shape[:-1])
    c_cs = 1 / M * np.sum((B * b).sum(-1) / vector_norm(B) / vector_norm(b))

    E_n = vector_norm(B - b).sum() / vector_norm(b).sum()

    E_m = 1 / M * (vector_norm(B - b) / vector_norm(b)).sum()

    eps = (vector_norm(B) ** 2).sum() / (vector_norm(b) ** 2).sum()

    # B_potential = get_potential(B[:, :, 0, 2], B.shape[-1])
    eps_p = (vector_norm(B) ** 2).sum() / (vector_norm(B_potential) ** 2).sum()

    # b_potential = get_potential(b[:, :, 0, 2], b.shape[-1])
    eps_p_b = (vector_norm(b) ** 2).sum() / (vector_norm(b_potential) ** 2).sum()

    j = curl(B)
    sig_J = (vector_norm(np.cross(j, B, -1)) / vector_norm(B)).sum() / vector_norm(j).sum()
    L1 = (vector_norm(np.cross(j, B, -1)) ** 2 / vector_norm(B) ** 2).mean()
    L2 = (divergence(B) ** 2).mean()

    # j_b = curl(b)
    # sig_J_b = (vector_norm(np.cross(j_b, b, -1)) / vector_norm(b)).sum() / vector_norm(j_b).sum()
    # L1_b = (vector_norm(np.cross(j_b, b, -1)) ** 2 / vector_norm(b) ** 2).mean()
    # L2_b = (divergence(b) ** 2).mean()

    key = ["C_vec", "C_cs", "1-En", "1-Em", "eps", "eps_p", "sig_J", "L1", "L2"]
    metric = [c_vec, c_cs, 1-E_n, 1-E_m, eps, eps_p, sig_J, L1, L2]
    return dict(zip(key, metric))

In [4]:
import os
import glob
import numpy as np

from pinf.analytical_field import get_analytic_b_field
from pinf.unpack import load_cube
from pinf.potential_field import get_potential

In [5]:
vtk_path = './output/vtk'
metric_path = './output/eval'
plot_path = './output/plot'
os.makedirs(vtk_path, exist_ok=True)
os.makedirs(metric_path, exist_ok=True)
os.makedirs(plot_path, exist_ok=True)

In [6]:
b = get_analytic_b_field(n=1, m=1, l=0.3, psi=np.pi/2, resolution=64, bounds=[-1, 1, -1, 1, 0, 2])

In [7]:
b_potential = get_potential(b[:, :, 0, 2], b.shape[-1])

Potential Field: 100%|██████████| 6/6 [00:00<00:00, 13.17it/s]


In [87]:
title = 'LL'
vtk_file = os.path.join(vtk_path, f'{title}.vtk')
p = pv_plot(B=b, vtk_path=vtk_file, points=((16, 49, 8), (16, 49, 8)), title=title)

p.camera_position = 'xy'
p.save_graphic(os.path.join(plot_path, f'{title}_xy.pdf'))
p.camera_position = 'yz'
p.save_graphic(os.path.join(plot_path, f'{title}_yz.pdf'))
p.camera_position = 'xz'
p.save_graphic(os.path.join(plot_path, f'{title}_xz.pdf'))
p.camera_position = 'xz'
p.camera.azimuth = -30
p.camera.elevation = 25
p.save_graphic(os.path.join(plot_path, f'{title}_xz_tilted.pdf'))

In [19]:
metric = metrics(B=b, b=b, B_potential=b_potential, b_potential=b_potential)
iterinfo = {'iter': -1}
metric = {**iterinfo, **metric}

In [20]:
df = pd.DataFrame.from_dict([metric])
df.to_csv(os.path.join(metric_path, 'metric.csv'), index=False)
df

,iter,C_vec,C_cs,1-En,1-Em,eps,eps_p,sig_J,L1,L2
0,-1,1.0,1.0,1.0,1.0,1.0,0.130138,0.01308,0.002065,0.002024


In [14]:
for file_path in sorted(glob.glob('run/dim256_bin1_pfTrue_ld0.1_lf0.1/fields_*.nf2')):
    iters = os.path.basename(file_path).split('.')[0][7:]
    title = 'PINN' + '_' + iters
    B = load_cube(file_path)
    
    # vtk_file = os.path.join(vtk_path, f'{title}.vtk')
    # p = pv_plot(B=B, vtk_path=vtk_file, points=((16, 49, 8), (16, 49, 8)), title=title)
    # p.camera_position = 'xy'
    # p.save_graphic(os.path.join(plot_path, f'{title}_xy.pdf'))
    # p.camera_position = 'yz'
    # p.save_graphic(os.path.join(plot_path, f'{title}_yz.pdf'))
    # p.camera_position = 'xz'
    # p.save_graphic(os.path.join(plot_path, f'{title}_xz.pdf'))
    # p.camera_position = 'xz'
    # p.camera.azimuth = -30
    # p.camera.elevation = 25
    # p.save_graphic(os.path.join(plot_path, f'{title}_xz_tilted.pdf'))

    metric = metrics(B=B, b=b, B_potential=b_potential, b_potential=b_potential)
    iterinfo = {'iter': int(iters)}
    metric = {**iterinfo, **metric}
    
    df = pd.concat([df, pd.DataFrame([metric])], ignore_index=True)

df.to_csv(os.path.join(metric_path, 'metric.csv'), index=False)
df

,iter,C_vec,C_cs,1-En,1-Em,eps,eps_p,sig_J,L1,L2
0,-1,1.000000,1.000000,1.000000,1.000000,1.000000,0.130138,0.013080,0.002065,0.002024
1,1000,0.608614,0.107008,-1.239649,-3.544160,3.704581,0.482106,0.741501,8.449643,3.628672
2,2000,0.737297,0.145278,-0.564850,-1.940714,2.738356,0.356364,0.631815,4.559499,4.849540
3,3000,0.790278,0.522197,-0.236990,-1.587884,1.761436,0.229230,0.678761,2.608789,2.282018
4,4000,0.857545,0.464909,0.111405,-0.764420,1.303401,0.169622,0.666579,1.630859,1.226921
5,5000,0.914070,0.471250,0.260928,-0.713482,1.079953,0.140543,0.623666,0.841915,0.627560
6,6000,0.939240,0.474954,0.380015,-0.280775,0.885824,0.115279,0.558254,0.424080,0.291443
7,7000,0.974462,0.315498,0.550867,-0.113525,0.917684,0.119425,0.435900,0.156409,0.111575
8,8000,0.986880,0.734498,0.689315,0.334515,0.954469,0.124213,0.346514,0.066033,0.046915
9,9000,0.987984,0.719207,0.705628,0.392912,0.965020,0.125586,0.302905,0.051924,0.031985
